In [40]:
import pandas as pd

In [41]:
data = pd.read_csv("./dsi_final_cleaned.TFs_HMs.txt", sep ='\t')

In [3]:
data[['Chromosome','Start','End']] = data['Location'].str.split(":|-", expand = True)

In [4]:
data['X']==True

0         False
1         False
2         False
3         False
4         False
          ...  
428059    False
428060    False
428061    False
428062    False
428063    False
Name: X, Length: 428064, dtype: bool

In [5]:
subset = data[data['Chromosome'] == "X"]
subset 


,Location,BEAF-2,CTCF-1,CondensinI-1,DREF-1,Ez-1,H1-1,H2Bub-1,H3-1,H3K4me2-1,...,Mod,POLS5P,PROseq,RNAseq,TFIIIC,lbf,X,Chromosome,Start,End
352206,X:0-300,-0.295,-1.701,2.851,-1.467,-0.162,-0.567,-0.492,-1.53,-0.277,...,-0.005,-0.354,-0.524,-0.052,-1.620,-0.834,True,X,0,300
352207,X:300-600,-0.295,-1.701,1.620,-1.467,-0.197,-0.567,-0.492,-1.53,-0.277,...,-0.003,-0.354,-0.524,-0.052,-1.620,-0.834,True,X,300,600
352208,X:600-900,-0.295,-1.701,0.641,-1.467,-0.193,-0.567,-0.492,-1.53,-0.277,...,-0.566,-0.354,-0.524,-0.052,-1.620,-0.834,True,X,600,900
352209,X:900-1200,-0.295,-1.701,0.159,-1.467,-0.164,-0.567,-0.492,-1.53,-0.277,...,-0.422,-0.354,-0.524,-0.051,-1.620,-0.834,True,X,900,1200
352210,X:1200-1500,-0.295,-1.701,-0.347,-1.467,-0.139,-0.567,-0.492,-1.53,-0.277,...,-0.326,-0.354,-0.524,-0.051,-1.620,-0.834,True,X,1200,1500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426641,X:22421400-22421700,-0.146,1.539,1.095,1.222,-0.059,-0.567,-0.492,-1.53,-0.277,...,0.033,-0.226,-0.503,-0.051,0.436,-0.834,True,X,22421400,22421700
426642,X:22421700-22422000,-0.155,1.499,-0.347,0.327,-0.194,-0.567,-0.492,-1.53,-0.277,...,0.274,-0.369,-5.546,-0.051,0.287,-0.834,True,X,22421700,22422000
426643,X:22422000-22422300,-0.043,1.061,-0.347,0.963,-0.208,-0.567,-0.492,-1.53,-0.277,...,-0.144,-0.356,-5.514,-0.051,0.469,-0.834,True,X,22422000,22422300
426644,X:22422300-22422600,-0.295,-1.222,-0.347,-1.174,0.328,-0.567,-0.492,-1.53,-0.277,...,-0.677,-0.380,-5.546,-0.052,-1.187,-0.834,True,X,22422300,22422600


In [6]:
subset =  subset.drop('CLAMP', axis=1)
subset.columns

Index(['Location', 'BEAF-2', 'CTCF-1', 'CondensinI-1', 'DREF-1', 'Ez-1',
       'H1-1', 'H2Bub-1', 'H3-1', 'H3K4me2-1', 'H3K9me3-1', 'H3_3-1',
       'H4K20me1-1', 'HA-1', 'L3mbt-1', 'M1BP-1', 'NELF-1', 'Nup98-1',
       'Pita-1', 'ZIPIC-1', 'dRing-1', 'ATAC_seq', 'Ash1', 'CBP', 'CP190',
       'Chromator', 'Cohesin', 'FAIRE', 'Fs1h', 'GAF', 'H3K27ac', 'H3K36me3',
       'H3K4me1', 'H3K9me2', 'H3k27me3', 'HP1', 'MACC', 'Mod', 'POLS5P',
       'PROseq', 'RNAseq', 'TFIIIC', 'lbf', 'X', 'Chromosome', 'Start', 'End'],
      dtype='object')

In [7]:
subset['Chromosome'] = 'chr'+subset['Chromosome']

In [8]:
# subset.reset_index()

In [9]:
bed_file = subset[['Chromosome','Start','End']]

In [10]:
bed_file.to_csv("X_TFs_HMs.bed", index = None, header = None, sep = '\t')

In [32]:
%%bash
module load bedtools
bedtools intersect -a X_TFs_HMs.bed -b dm6_ces1_2.bed -f 0.6 -loj > X_merged_labels.bed 

module: loading 'bedtools/2.31.0'


In [33]:
labels = pd.read_csv("X_merged_labels.bed", sep ="\t", header = None)

In [34]:
labels

,0,1,2,3,4,5,6
0,chrX,0,300,.,-1,-1,.
1,chrX,300,600,.,-1,-1,.
2,chrX,600,900,.,-1,-1,.
3,chrX,900,1200,.,-1,-1,.
4,chrX,1200,1500,.,-1,-1,.
...,...,...,...,...,...,...,...
74435,chrX,22421400,22421700,.,-1,-1,.
74436,chrX,22421700,22422000,.,-1,-1,.
74437,chrX,22422000,22422300,.,-1,-1,.
74438,chrX,22422300,22422600,.,-1,-1,.


In [35]:
condition = (labels[6] !='.')

In [36]:
from collections import Counter
Counter(condition)

Counter({False: 74072, True: 368})

In [37]:
import numpy as np

In [38]:
subset['label'] =  np.where(condition, 1, 0)


In [39]:
subset = subset.reset_index(drop = True)

In [19]:
subset.to_csv("subset_X_labels.tsv", sep ="\t", header = True, index = False)

In [20]:
subset_nolabel = np.array(subset.iloc[:, 1:-5])

In [21]:
# Define the number of rows you want in each submatrix
num_rows_per_submatrix = 41

# Extract 50 consecutive rows as submatrices
submatrices = np.stack([subset_nolabel[i:i+num_rows_per_submatrix, :] for i in range(0, subset_nolabel.shape[0]-num_rows_per_submatrix)])

labels_subset = subset['label'][20:-21]

labels_subset.reset_index(drop = True)

0        0
1        0
2        0
3        0
4        0
        ..
74394    0
74395    0
74396    0
74397    0
74398    0
Name: label, Length: 74399, dtype: int64

In [22]:
### matrix of all data samples 

submatrices.shape

(74399, 41, 42)

In [23]:
data = np.asarray(submatrices)

In [24]:
full_data = pd.read_csv("subset_X_labels.tsv", sep ="\t")
labels = full_data['label']
labels = np.asarray(labels)[:-num_rows_per_submatrix]
labels.shape

(74399,)

In [25]:
# positive_indices = np.where(labels == 1)[0]
# new_labels = np.zeros(len(labels))

# for pos_i in positive_indices:
#     new_labels[pos_i:pos_i+52] = 1

In [31]:
data.shape

(74399, 41, 42)

In [26]:
data.shape, labels.shape

((74399, 41, 42), (74399,))

In [27]:
# Full Dataset Exporting
np.save("X_data.npy", data)
np.save("X_labels.npy", labels)

In [28]:
from collections import Counter
Counter(labels)

Counter({0: 74126, 1: 273})

## Sample Small Toy Dataset to Run Locally for Model Verification

In [29]:
## Sample 3000 data points from the full dataset
import random
random.seed(123)

sampled_indices = random.sample(range(0, len(data)), 3000)
sampled_data = data[sampled_indices, :, :]
sampled_labels = new_labels[sampled_indices]

# Sampled Dataset Exporting
np.save("X_sampled_data.npy", sampled_data)
np.save("X_sampled_labels.npy", sampled_labels)

NameError: name 'new_labels' is not defined

In [ ]:
Counter(data['Chromosome'].to_list())

In [ ]:
grouped = data.groupby('Chromosome')


In [ ]:
for category,group in grouped:
    print(category)
    print#(group)
    print(Counter(group['X']))